In [38]:
parent_dir = "C:/Users/Davide/git/gitdf/EnergyCommunity.jl/run_cloud"

n_iter = 2
EC_size_list_enum = [3,4,5] #[5, 10, 20]  # List of sizes of the EC to test in enum mode

3-element Vector{Int64}:
 3
 4
 5

In [33]:
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML

## Compare ENUM_MODE

In [34]:
dict_enums = Dict(
    size_enum=>load("$parent_dir/results_paper/enum/enum_simulations_results_$size_enum.jld2")
    for size_enum in EC_size_list_enum
)

Dict{Int64, Dict{String, Any}} with 3 entries:
  5 => Dict("df_reward_enum"=>6×5 DataFrame…
  4 => Dict("df_reward_enum"=>5×5 DataFrame…
  3 => Dict("df_reward_enum"=>4×5 DataFrame…

### ENUM_MODE: Create reward redistribution of enum modes

In [35]:
df_reward_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_reward_temp = deepcopy(dict_enums[EC_size]["df_reward_enum"])
    reward_list = setdiff(names(df_reward_temp), ["user_set"])
    rename!(df_reward_temp, reward_list .=> reward_list .* "_$EC_size")
    if nrow(df_reward_enum) == 0
        df_reward_enum = df_reward_temp
    else
        df_reward_enum = outerjoin(df_reward_enum, df_reward_temp, on=:user_set, makeunique=true)
    end
end

# sort by user set
sort!(df_reward_enum, :user_set)

df_reward_enum

,user_set,shapley_enum_3,nucleolus_enum_3,varcore_enum_3,varleastcore_enum_3
,String,Float64?,Float64?,Float64?,Float64?
1,EC,3026.19,3364.02,4062.18,3364.02
2,user1,5333.46,5957.57,4062.18,5758.51
3,user2,3612.78,3364.02,4062.18,3364.02
4,user3,4276.28,3563.09,4062.18,3762.15
5,user4,missing,missing,missing,missing
6,user5,missing,missing,missing,missing


### ENUM_MODE: Create comparison of computational time

In [36]:
df_time_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_time_temp = DataFrame(dict_enums[EC_size]["df_time_enum"])  # dict_time_enum

    # df_time_temp[!, 2:end] ./= 3600  # change units to hours

    df_time_temp[!, "EC size"] = [EC_size]
    df_time_temp[!, "title"] = [L"Time [h]"]
    df_time_temp = df_time_temp[!, ["EC size"; "title"; names(df_time_temp)[1:end-2]]]

    if nrow(df_time_enum) == 0
        df_time_enum = df_time_temp
    else
        df_time_enum = vcat(df_time_enum, df_time_temp)
    end
end

df_time_enum

,EC size,title,name,id_run,EC_size,mode_time,shapley_enum,nucleolus_enum
,Int64,LaTeXStr…,String,Float64,Int64,Float64,Float64,Float64
1,3,$Time [h]$,enum_mode,0.0,3,8.2594,8.3987,9.48579
2,4,$Time [h]$,enum_mode,0.0,4,13.5661,13.5671,13.608
3,5,$Time [h]$,enum_mode,0.0,5,33.464,33.4647,33.5301


## Compare ITER_MODE

In [40]:
dict_iter = Dict(
    id_run=>load("$parent_dir/results_paper/iter/iter_simulations_results_$id_run.jld2")
    for id_run in 1:n_iter
)

Dict{Int64, Dict{String, Any}} with 2 entries:
  2 => Dict("df_reward_iter"=>5×5 DataFrame…
  1 => Dict("df_reward_iter"=>4×5 DataFrame…

#### ITER_MODE: Comparison of reward distribution

In [43]:
df_reward_iter = DataFrame()

for id_run in 1:n_iter
    df_reward_temp = deepcopy(dict_iter[id_run]["df_reward_iter"])
    reward_list = setdiff(names(df_reward_temp), ["user_set"])
    rename!(df_reward_temp, reward_list .=> reward_list .* "_$(nrow(df_reward_temp)-1)")
    if nrow(df_reward_iter) == 0
        df_reward_iter = df_reward_temp
    else
        df_reward_iter = outerjoin(df_reward_iter, df_reward_temp, on=:user_set, makeunique=true)
    end
end

# sort by user set
sort!(df_reward_iter, :user_set)

df_reward_iter

,user_set,incore_iter_3,leastcore_iter_3,varcore_iter_3,varleastcore_iter_3
,String,Float64?,Float64?,Float64?,Float64?
1,EC,7623.01,3364.02,4062.18,3364.02
2,user1,6288.13,6156.63,4062.18,5758.51
3,user2,2337.57,3364.02,4062.18,3364.02
4,user3,0.0,3364.02,4062.18,3762.15
5,user4,missing,missing,missing,missing


#### ITER_MODE: Create comparison of computational time

In [47]:
dict_iter[1]["df_time_iter"]

,name,id_run,EC_size,mode_time,incore_iter,leastcore_iter,varcore_iter
,String,Int64,Int64,Float64,Float64,Float64,Float64
1,iter_mode,1,3,0.0,48.6476,59.9421,27.2023


In [51]:
df_time_iter = DataFrame()

for id_run in 1:n_iter
    df_time_temp = deepcopy(dict_iter[id_run]["df_time_iter"])

    # df_time_temp[!, 2:end] ./= 3600  # change units to hours

    df_time_temp[!, "title"] = [L"Time [h]"]

    if nrow(df_time_iter) == 0
        df_time_iter = df_time_temp
    else
        df_time_iter = vcat(df_time_iter, df_time_temp)
    end
end

header_cols = ["name", "EC_size", "id_run", "title"]
df_list_rewards = setdiff(names(df_time_iter), header_cols)

df_time_iter = df_time_iter[!, [header_cols; df_list_rewards]]

,name,EC_size,id_run,title,mode_time,incore_iter,leastcore_iter
,String,Int64,Int64,LaTeXStr…,Float64,Float64,Float64
1,iter_mode,3,1,$Time [h]$,0.0,48.6476,59.9421
2,iter_mode,4,2,$Time [h]$,0.0,150.975,91.2564
